In [17]:
import pandas as pd

In [46]:
timeline = './flight_logs/forensic_timeline.csv'
timeline = pd.read_csv(timeline, encoding= 'unicode_escape')
timeline

,date,time,message
0,06/19/2018,15:33:55,Weak GPS signal. Aircraft is in Attitude mode ...
1,06/19/2018,15:34:06,Weak GPS signal. Aircraft is in Attitude mode ...
2,06/19/2018,15:34:25,Weak GPS signal. Aircraft is in Attitude mode ...
3,06/19/2018,15:34:39,Home Point recorded. Return-to-Home Altitude:98FT
4,06/19/2018,15:34:59,Home Point recorded. Return-to-Home Altitude:98FT
...,...,...,...
168,06/19/2018,14:51:37,Fly with caution and ensure the aircraft remai...
169,06/19/2018,14:52:01,High wind velocity. Fly with caution and land ...
170,06/19/2018,14:52:17,Fly with caution and ensure the aircraft remai...
171,06/19/2018,14:52:29,Fly with caution and ensure the aircraft remai...


In [47]:
timeline.columns

Index(['date', 'time', 'message'], dtype='object')

In [48]:
# making copy of team column
time = timeline["time"].copy()
 
# concatenating team with name column
# overwriting name column
timeline["timestamp"] = timeline["date"].str.cat(time, sep =" ")
timeline.drop(columns=['time', 'date'], inplace=True)
timeline = timeline[['timestamp', 'message']]
# display
timeline

,timestamp,message
0,06/19/2018 15:33:55,Weak GPS signal. Aircraft is in Attitude mode ...
1,06/19/2018 15:34:06,Weak GPS signal. Aircraft is in Attitude mode ...
2,06/19/2018 15:34:25,Weak GPS signal. Aircraft is in Attitude mode ...
3,06/19/2018 15:34:39,Home Point recorded. Return-to-Home Altitude:98FT
4,06/19/2018 15:34:59,Home Point recorded. Return-to-Home Altitude:98FT
...,...,...
168,06/19/2018 14:51:37,Fly with caution and ensure the aircraft remai...
169,06/19/2018 14:52:01,High wind velocity. Fly with caution and land ...
170,06/19/2018 14:52:17,Fly with caution and ensure the aircraft remai...
171,06/19/2018 14:52:29,Fly with caution and ensure the aircraft remai...


In [49]:
# checking datatype
print(type(timeline.timestamp[0]))
 
# convert to date
timeline['timestamp'] = pd.to_datetime(timeline['timestamp'])
 
# verify datatype
print(type(timeline.timestamp[0]))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [12]:
timeline

,date,time,message
0,2018-06-19 15:33:55,15:33:55,Weak GPS signal. Aircraft is in Attitude mode ...
1,2018-06-19 15:34:06,15:34:06,Weak GPS signal. Aircraft is in Attitude mode ...
2,2018-06-19 15:34:25,15:34:25,Weak GPS signal. Aircraft is in Attitude mode ...
3,2018-06-19 15:34:39,15:34:39,Home Point recorded. Return-to-Home Altitude:98FT
4,2018-06-19 15:34:59,15:34:59,Home Point recorded. Return-to-Home Altitude:98FT
...,...,...,...
168,2018-06-19 14:51:37,14:51:37,Fly with caution and ensure the aircraft remai...
169,2018-06-19 14:52:01,14:52:01,High wind velocity. Fly with caution and land ...
170,2018-06-19 14:52:17,14:52:17,Fly with caution and ensure the aircraft remai...
171,2018-06-19 14:52:29,14:52:29,Fly with caution and ensure the aircraft remai...


In [51]:
# sorting DataFrame by giving first priority to Expiry Date and then Mfg. Date
timeline.sort_values(by='timestamp', inplace=True)

In [52]:
timeline

,timestamp,message
101,2018-06-15 11:07:46,"Aircraft is in Attitude mode, so that it will ..."
102,2018-06-15 11:08:00,"Aircraft is in Attitude mode, so that it will ..."
103,2018-06-15 11:08:02,"Aircraft is in Attitude mode, so that it will ..."
104,2018-06-15 11:08:15,"Aircraft is in Attitude mode, so that it will ..."
105,2018-06-15 11:08:19,Cannot Takeoff in Travel Mode. Exit Travel Mode.
...,...,...
1,2018-06-19 15:34:06,Weak GPS signal. Aircraft is in Attitude mode ...
2,2018-06-19 15:34:25,Weak GPS signal. Aircraft is in Attitude mode ...
3,2018-06-19 15:34:39,Home Point recorded. Return-to-Home Altitude:98FT
4,2018-06-19 15:34:59,Home Point recorded. Return-to-Home Altitude:98FT


In [53]:
timeline.to_csv('./flight_logs/forensic_timeline_sorted.csv', index=False, encoding="utf-8") 

In [54]:
# Load the NER model
from simpletransformers.ner import NERModel

droner = NERModel("bert", "outputs", use_cuda=True)

In [64]:
message = timeline.iloc[0,1]
message
[entities], _ = droner.predict(["Aircraft is in Attitude mode, so that it will not hover. Please fly with caution."])
entities

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'Aircraft': 'O'},
 {'is': 'O'},
 {'in': 'O'},
 {'Attitude': 'O'},
 {'mode,': 'O'},
 {'so': 'O'},
 {'that': 'O'},
 {'it': 'O'},
 {'will': 'O'},
 {'not': 'O'},
 {'hover.': 'O'},
 {'Please': 'O'},
 {'fly': 'O'},
 {'with': 'O'},
 {'caution.': 'O'}]

In [70]:
pred_list = []
for row in range(0, timeline.shape[0]):
    message = timeline.iloc[row,1]
    [entities], _ = droner.predict([message])
    pred_list.append({'timestamp': string(timeline.iloc[row, 0]), 'entities': entities})

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
pred_list

[{'timestamp': Timestamp('2018-06-15 11:07:46'),
  'entities': [{'Aircraft': 'O'},
   {'is': 'O'},
   {'in': 'O'},
   {'Attitude': 'O'},
   {'mode,': 'O'},
   {'so': 'O'},
   {'that': 'O'},
   {'it': 'O'},
   {'will': 'O'},
   {'not': 'O'},
   {'hover.': 'O'},
   {'Please': 'O'},
   {'fly': 'O'},
   {'with': 'O'},
   {'caution.': 'O'}]},
 {'timestamp': Timestamp('2018-06-15 11:08:00'),
  'entities': [{'Aircraft': 'O'},
   {'is': 'O'},
   {'in': 'O'},
   {'Attitude': 'O'},
   {'mode,': 'O'},
   {'so': 'O'},
   {'that': 'O'},
   {'it': 'O'},
   {'will': 'O'},
   {'not': 'O'},
   {'hover.': 'O'},
   {'Please': 'O'},
   {'fly': 'O'},
   {'with': 'O'},
   {'caution.': 'O'}]},
 {'timestamp': Timestamp('2018-06-15 11:08:02'),
  'entities': [{'Aircraft': 'O'},
   {'is': 'O'},
   {'in': 'O'},
   {'Attitude': 'O'},
   {'mode,': 'O'},
   {'so': 'O'},
   {'that': 'O'},
   {'it': 'O'},
   {'will': 'O'},
   {'not': 'O'},
   {'hover.': 'O'},
   {'Please': 'O'},
   {'fly': 'O'},
   {'with': 'O'},
   {

In [69]:
print(len(pred_list))

173


In [78]:
for predict in pred_list:
    print(predict['timestamp'])
    print(predict['entities'])
    for dic in predict['entities']:
        for token, tag in dic.items():
#             for word, tag in token:
            print('word: {word}, tag: {tag}'.format(word=token, tag=tag))

2018-06-15 11:07:46
[{'Aircraft': 'O'}, {'is': 'O'}, {'in': 'O'}, {'Attitude': 'O'}, {'mode,': 'O'}, {'so': 'O'}, {'that': 'O'}, {'it': 'O'}, {'will': 'O'}, {'not': 'O'}, {'hover.': 'O'}, {'Please': 'O'}, {'fly': 'O'}, {'with': 'O'}, {'caution.': 'O'}]
word: Aircraft, tag: O
word: is, tag: O
word: in, tag: O
word: Attitude, tag: O
word: mode,, tag: O
word: so, tag: O
word: that, tag: O
word: it, tag: O
word: will, tag: O
word: not, tag: O
word: hover., tag: O
word: Please, tag: O
word: fly, tag: O
word: with, tag: O
word: caution., tag: O
2018-06-15 11:08:00
[{'Aircraft': 'O'}, {'is': 'O'}, {'in': 'O'}, {'Attitude': 'O'}, {'mode,': 'O'}, {'so': 'O'}, {'that': 'O'}, {'it': 'O'}, {'will': 'O'}, {'not': 'O'}, {'hover.': 'O'}, {'Please': 'O'}, {'fly': 'O'}, {'with': 'O'}, {'caution.': 'O'}]
word: Aircraft, tag: O
word: is, tag: O
word: in, tag: O
word: Attitude, tag: O
word: mode,, tag: O
word: so, tag: O
word: that, tag: O
word: it, tag: O
word: will, tag: O
word: not, tag: O
word: hover.